In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Pipelines：使用google-cloud-pipeline-components的AutoML文本分类管道

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/google_cloud_pipeline_components_automl_text.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/google_cloud_pipeline_components_automl_text.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/pipelines/google_cloud_pipeline_components_automl_text.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
     </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本笔记本展示如何使用[`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud)中定义的组件，在[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines)上构建一个AutoML文本分类工作流程。

了解更多关于[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)和[AutoML组件](https://cloud.google.com/vertex-ai/docs/pipelines/vertex-automl-component)。

### 目标

在本教程中，您将学习如何使用`Vertex AI Pipelines`和`Google Cloud Pipeline Components`来构建一个`AutoML`文本分类模型。

本教程使用以下Google Cloud ML服务：

- `Vertex AI Pipelines`
- `Google Cloud Pipeline Components`
- `Vertex AutoML`
- `Vertex AI Model`资源
- `Vertex AI Endpoint`资源

执行的步骤包括：

- 创建一个KFP管道：
    - 创建一个`Dataset`资源。
    - 训练一个AutoML文本分类的`Model`资源。
    - 创建一个`Endpoint`资源。
    - 将`Model`资源部署到`Endpoint`资源。
- 编译KFP管道。
- 使用`Vertex AI Pipelines`执行KFP管道。

组件的文档在[这里](https://google-cloud-pipeline-components.readthedocs.io/en/latest/google_cloud_pipeline_components.aiplatform.html#module-google_cloud_pipeline_components.aiplatform)。

数据集

本教程使用的数据集是来自Kaggle数据集的[幸福时刻数据集]（https://www.kaggle.com/ritresearch/happydb）。您将在本教程中使用的数据集版本存储在公共云存储桶中。

### 费用

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 基于您的预期使用生成费用估算。

## 安装

安装执行这个笔记本所需的软件包。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 kfp \
                                 google-cloud-pipeline-components

### 仅限Colab：取消下面的单元格注释以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的Google Cloud项目

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建一个账户时，您将获得$300免费信用额度用于计算/存储成本。

2. [确保为您的项目启用了结算功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API]。

4. 如果您是在本地运行这个笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

#### 设置您的项目 ID

 **如果您不知道您的项目 ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 认证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动认证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 无需操作，因为您已经认证过。

**2. 本地JupyterLab实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

请看如何在 https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 给你的服务帐号授予云存储权限。

创建一个云存储桶，用于存储中间产物，如数据集。

只有在您的存储桶不存在时才运行以下单元格来创建您的云存储存储桶。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时，运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

服务账号

**如果您不知道您的服务账号**，请尝试使用`gcloud`命令在下面执行第二个单元格来获取您的服务账号。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import os
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

将服务账户访问权设置为 Vertex AI Pipelines

运行以下命令，将您的服务账户访问权授予读取和写入管道工件的权限，此桶是您在前一步骤中创建的 - 您只需要每个服务账户运行一次这些命令。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 设置变量

接下来，设立一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip
import kfp

### Vertex AI Pipelines 常量

为 Vertex AI Pipelines 设置以下常量：

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/happydb".format(BUCKET_URI)

## 初始化 Python 的 Vertex AI SDK

为您的项目和对应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

定义一个使用`google_cloud_pipeline_components`组件的AutoML文本分类模型管道

接下来，你需要定义管道。

使用`Dataset`资源创建并部署一个AutoML文本分类`Model`资源。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-data/NL-classification/happiness.csv"


@kfp.dsl.pipeline(name="automl-text-classification")
def pipeline(
    project: str = PROJECT_ID, region: str = REGION, import_file: str = IMPORT_FILE
):
    from google_cloud_pipeline_components.v1.automl.training_job import \
        AutoMLTextTrainingJobRunOp
    from google_cloud_pipeline_components.v1.dataset import TextDatasetCreateOp
    from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,
                                                              ModelDeployOp)

    dataset_create_task = TextDatasetCreateOp(
        display_name="train-automl-happydb",
        gcs_source=import_file,
        import_schema_uri=aip.schema.dataset.ioformat.text.multi_label_classification,
        project=project,
    )

    training_run_task = AutoMLTextTrainingJobRunOp(
        dataset=dataset_create_task.outputs["dataset"],
        display_name="train-automl-happydb",
        prediction_type="classification",
        multi_label=True,
        training_fraction_split=0.6,
        validation_fraction_split=0.2,
        test_fraction_split=0.2,
        model_display_name="train-automl-happydb",
        project=project,
    )

    endpoint_op = EndpointCreateOp(
        project=project,
        location=region,
        display_name="train-automl-flowers",
    )

    ModelDeployOp(
        model=training_run_task.outputs["model"],
        endpoint=endpoint_op.outputs["endpoint"],
        automatic_resources_min_replica_count=1,
        automatic_resources_max_replica_count=1,
    )

编译这个流程。

In [ ]:
from kfp.v2 import compiler  # noqa: F811

compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="text_classification_pipeline.yaml",
)

## 运行管道

接下来，运行管道。

In [ ]:
DISPLAY_NAME = "happydb"

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="text_classification_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
)

job.run()

! rm text_classification_pipeline.yaml

点击生成的链接来查看在云控制台中运行的内容。

在UI中，当您点击它们时，许多管道DAG节点将展开或折叠。这是DAG的部分展开视图（点击图像查看更大版本）。

<a href="https://storage.googleapis.com/amy-jo/images/mp/automl_text_classif.png" target="_blank"><img src="https://storage.googleapis.com/amy-jo/images/mp/automl_text_classif.png" width="40%"/></a>

# 清理

要清理这个项目中使用的所有Google Cloud资源，您可以删除用于该教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源 -- *提示:* 这是自动生成的，不是所有资源都适用于此教程。

In [ ]:
delete_bucket = False

try:
    if "DISPLAY_NAME" in globals():
        models = aip.Model.list(
            filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
        )
        model = models[0]
        aip.Model.delete(model)
        print("Deleted model:", model)
except Exception as e:
    print(e)

try:
    if delete_endpoint and "DISPLAY_NAME" in globals():
        endpoints = aip.Endpoint.list(
            filter=f"display_name={DISPLAY_NAME}_endpoint", order_by="create_time"
        )
        endpoint = endpoints[0]
        endpoint.undeploy_all()
        aip.Endpoint.delete(endpoint.resource_name)
        print("Deleted endpoint:", endpoint)
except Exception as e:
    print(e)

if "DISPLAY_NAME" in globals():

    try:
        datasets = aip.TextDataset.list(
            filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
        )
        dataset = datasets[0]
        aip.TextDataset.delete(dataset.resource_name)
        print("Deleted dataset:", dataset)
    except Exception as e:
        print(e)

try:
    if "DISPLAY_NAME" in globals():
        pipelines = aip.PipelineJob.list(
            filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
        )
        pipeline = pipelines[0]
        aip.PipelineJob.delete(pipeline.resource_name)
        print("Deleted pipeline:", pipeline)
except Exception as e:
    print(e)

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI